In [161]:
import findspark
#初始化
findspark.init()
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql import SparkSession



In [162]:
# 定义数据库的地址，以及表，登录用户及密码
url = "jdbc:mysql://localhost:3306/sxzy"
table="df_gongzi"
#密码账户需要字典的形式传入
properties ={"user":"root","password":"521521"}
spark = SparkSession.builder.appName('My first app').getOrCreate()
df = spark.read.jdbc(url=url,table=table,properties=properties)
# df.show(4)


In [163]:
df  # 查看字段，通过字段回去需要的数据

DataFrame[cxkh: string, jymc: string, jdbz: string, jysj: string, jyje: string, jybz: string, jydfzkh: string, jydfmc: string, ymd: string]

In [164]:
df.count(),len(df.columns)# 查看数据维度

(21136, 9)

In [165]:
df.printSchema()# 元数据分析，查看字段类型

root
 |-- cxkh: string (nullable = true)
 |-- jymc: string (nullable = true)
 |-- jdbz: string (nullable = true)
 |-- jysj: string (nullable = true)
 |-- jyje: string (nullable = true)
 |-- jybz: string (nullable = true)
 |-- jydfzkh: string (nullable = true)
 |-- jydfmc: string (nullable = true)
 |-- ymd: string (nullable = true)



In [166]:
df1=df.select(['cxkh','jymc']) # 按照列名选择

In [167]:
# df1.show(5)

In [168]:
df2=df.filter((df['jyje']>100000) & (df['jysj']>'2015-01-01')) #按照条件进行筛选

In [169]:
df.printSchema()

root
 |-- cxkh: string (nullable = true)
 |-- jymc: string (nullable = true)
 |-- jdbz: string (nullable = true)
 |-- jysj: string (nullable = true)
 |-- jyje: string (nullable = true)
 |-- jybz: string (nullable = true)
 |-- jydfzkh: string (nullable = true)
 |-- jydfmc: string (nullable = true)
 |-- ymd: string (nullable = true)



In [170]:
df3=df.select(['jyje'])
# df3.show(3)

In [171]:
df3.printSchema()

root
 |-- jyje: string (nullable = true)



In [172]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

In [173]:
# 类型转换
df3 = df3.withColumn("jyje",df3.jyje.astype("int"))

In [174]:
# 生成密集性向量
assembler = VectorAssembler(inputCols=df3.columns,outputCol="features")
df_new=assembler.transform(df3)

In [175]:
df_new.show(2)

+----+--------+
|jyje|features|
+----+--------+
|6000|[6000.0]|
|1500|[1500.0]|
+----+--------+
only showing top 2 rows



In [176]:
Correlation.corr(df_new,'features').show()## pearson相关性

+-----------------+
|pearson(features)|
+-----------------+
|            1.0  |
+-----------------+



In [177]:
Correlation.corr(df_new,'features', 'spearman').show()# 计算spearman系数

+------------------+
|spearman(features)|
+------------------+
|             1.0  |
+------------------+



In [178]:
df3.describe().show()  # 描述性统计

+-------+------------------+
|summary|              jyje|
+-------+------------------+
|  count|             21136|
|   mean|158311.19081188494|
| stddev|1335809.6361363924|
|    min|              1000|
|    max|          61726798|
+-------+------------------+



In [179]:
df3.printSchema()#查看字段属性

root
 |-- jyje: integer (nullable = true)



In [182]:
df.filter(df['jymc']=='xx').show(2)  # 样本选择

+----+----+----+----+----+----+-------+------+---+
|cxkh|jymc|jdbz|jysj|jyje|jybz|jydfzkh|jydfmc|ymd|
+----+----+----+----+----+----+-------+------+---+
+----+----+----+----+----+----+-------+------+---+



In [183]:
df.filter(df['jymc']=='xx').select('cxkh','jdbz','jyje').show(2) # 样本+变量选择

+----+----+----+
|cxkh|jdbz|jyje|
+----+----+----+
+----+----+----+



In [184]:
df.withColumn('test',(df['jyje']+100)).select('jyje','test').show(5) # 增加字段

+-------+-------+
|   jyje|   test|
+-------+-------+
| 6000.0| 6100.0|
| 1500.0| 1600.0|
| 2000.0| 2100.0|
| 6000.0| 6100.0|
|10000.0|10100.0|
+-------+-------+
only showing top 5 rows



In [185]:
# df.select('zysm').show()
ddd=df[df['jydfmc'].isNull()]  # 筛选未空

In [186]:
ddd1=ddd['jymc','jydfmc']

In [187]:
ddd1.fillna('wz').show(2) #缺失值填充
ddd1.na.fill('wz').show(2)  #缺失值填充

+------+------+
|  jymc|jydfmc|
+------+------+
|  刘兵|    wz|
|白翠平|    wz|
+------+------+
only showing top 2 rows

+------+------+
|  jymc|jydfmc|
+------+------+
|  刘兵|    wz|
|白翠平|    wz|
+------+------+
only showing top 2 rows



分组处理

In [188]:
df.groupBy('jymc').count().show(3)
df.groupBy('jymc').count().orderBy('count',ascending=False).show(3)

+------+-----+
|  jymc|count|
+------+-----+
|张丽丽|  453|
|高明明| 1075|
|吴晓虎|  233|
+------+-----+
only showing top 3 rows

+------+-----+
|  jymc|count|
+------+-----+
|  刘涛| 8141|
|张莉莉| 2598|
|高明明| 1075|
+------+-----+
only showing top 3 rows



In [189]:
df.groupBy('jyje').mean().show(5,False)

+-------+
|jyje   |
+-------+
|1664.34|
|1259.04|
|1488.74|
|1368.63|
|1857.14|
+-------+
only showing top 5 rows



In [190]:
df.groupBy('jymc','cxkh').agg({'jyje':'sum','jysj':'max'}).orderBy('sum(jyje)',ascending=False).show(5,False) #特定函数聚合求解

+------+-------------------+-------------------+--------------------+
|jymc  |cxkh               |max(jysj)          |sum(jyje)           |
+------+-------------------+-------------------+--------------------+
|崔新平|6229980707666660   |2019-11-22 15:32:39|7.1646981569E8      |
|崔新平|6217680705469992   |2019-10-23 10:23:24|2.4818253972000003E8|
|张丽丽|6229985520053194   |2019-03-01 16:10:24|2.0568639223999998E8|
|吴晓虎|6217680706701815   |2019-12-09 10:42:21|2.0276562496000004E8|
|杨晓龙|6222625430000169209|2019-12-14 10:48:07|1.8387798923999998E8|
+------+-------------------+-------------------+--------------------+
only showing top 5 rows



In [191]:
# df.select(['jymc','jydfmc']).stat.crosstab('jymc','jydfmc').show()
df.select(['cxkh','jymc']).dropDuplicates().show(5),
df.select(['cxkh','jymc']).dropDuplicates().count()

+-------------------+------+
|               cxkh|  jymc|
+-------------------+------+
|   6217685500292663|  刘涛|
|0509256802204209296|杨建进|
|   6216921002523698|杨建进|
|   6226223903387994|杨晓龙|
|       147970660363|郭凤兰|
+-------------------+------+
only showing top 5 rows



219

In [192]:
def jyje_level(x):
    if x< 100000 :
        return '一等'
    elif 100000<=x <500000:
        return '二等'
    else :
        return '三等'

In [193]:
from pyspark.sql.functions import  udf
from pyspark.sql.types import  StringType  #类似于ｌａｍｂｄａ
brand_udf=udf(jyje_level,StringType())  #注册ｕｄf
df3.withColumn('jyje_level',brand_udf(df3['jyje'])).show(5)   
brand_udf1=udf(lambda x: '+' if x>1000000 else '-',StringType())  #匿名函数
df3.withColumn('jyje_level1',brand_udf1(df3['jyje'])).show(5)


+-----+----------+
| jyje|jyje_level|
+-----+----------+
| 6000|      一等|
| 1500|      一等|
| 2000|      一等|
| 6000|      一等|
|10000|      一等|
+-----+----------+
only showing top 5 rows

+-----+-----------+
| jyje|jyje_level1|
+-----+-----------+
| 6000|          -|
| 1500|          -|
| 2000|          -|
| 6000|          -|
|10000|          -|
+-----+-----------+
only showing top 5 rows



In [194]:
df_like=df.select(['cxkh','jymc','jdbz','jyje','jydfmc']).filter("jydfmc rlike  '基金|理财|证券'") # 模糊匹配

In [198]:
# df_like.show(3),df_like.count(),len(df_like.columns)

In [199]:
df_person=df.select('cxkh','jymc').dropDuplicates()
# df_person.show(3)

In [200]:
df_lastime=df.select('cxkh','jysj','jyje').groupBy('cxkh').agg({'jysj':'max','jyje':'max'}).\
withColumnRenamed('cxkh','cxkh1').withColumnRenamed('max(jysj)','最晚一次交易时间').\
withColumnRenamed('max(jyje)','最大交易金额') #修改指定字段的名字， df.seleceExpr('first_name as lase_name'),这个好像需要字段名全部写上，否则最终显示结果只有更改了列名的那一列。
# df_lastime.show(2)

In [201]:
df_lastime.printSchema()

root
 |-- cxkh1: string (nullable = true)
 |-- 最晚一次交易时间: string (nullable = true)
 |-- 最大交易金额: string (nullable = true)



In [202]:
df_person.count(),df_lastime.count()

(219, 219)

In [203]:
df_person_info_lasttime=df_person.join(df_lastime,df_person['cxkh']==df_lastime['cxkh1'],\
                                       'left_outer').orderBy('cxkh',ascending=True)# 两表拼接指定字段的拼接，其他的连接方式：inner, outer, left_outer, right_outer, leftsemi

In [206]:
df_person_info_lasttime.registerTempTable('df_person_info_lasttime') # 将数据框注册为一个table
spark.sql("select * from df_person_info_lasttime  where jymc='xx'").show()#程序 mysql程序

+----+----+-----+----------------+------------+
|cxkh|jymc|cxkh1|最晚一次交易时间|最大交易金额|
+----+----+-----+----------------+------------+
+----+----+-----+----------------+------------+



In [81]:
#写出在多个csv文件中，一个csv文件为一行数据
df_person_info_lasttime.write.save('c:/users/user1/desktop/tt1.csv')

In [85]:
#写出在一个csv 文件中
df_person_info_lasttime.repartition(1).write.csv("path.csv",header=True, sep=",", mode='overwrite',encoding='utf-8')


In [86]:
#转换python pandas dataframe,,通过to_excel (),to_csv（）输出
d1=df_person_info_lasttime.toPandas() #转换成pandas ,可以用to_excel 输出

In [91]:
#保存为parquet格式数据。读取速度快，占用内存小
df_person_info_lasttime.write.format('parquet').save('path4.parquet')

In [93]:
#读取parquet数据
d1=spark.read.format('parquet').load('path4.parquet')


In [208]:
df_intersect=df.select('jymc').dropDuplicates().intersect(df.select('jydfmc').dropDuplicates())#交集
# df_intersect.count(),df_intersect.show(3)

In [209]:
df_subtract=df.select('jymc').dropDuplicates().subtract(df.select('jydfmc').dropDuplicates())#查缉
# df_subtract.count(),df_subtract.show(3)

In [210]:
df_union=df.select('jymc').dropDuplicates().union(df.select('jydfmc').dropDuplicates())#并缉
df_union.count(),df_union.show(3)

+------+
|  jymc|
+------+
|张丽丽|
|高明明|
|吴晓虎|
+------+
only showing top 3 rows



(1071, None)

In [211]:
df_union_distinct=df.select('jymc').union(df.select('jydfmc')).distinct()#并集+去重
# df_union_distinct.count(),df_union_distinct.show(3)

In [212]:
df_year=df.select('jysj')

In [213]:
from pyspark.sql.functions import split
df_year=df_year.withColumn('jyrq',split(df_year['jysj']," ").getItem(0))
df_year=df_year.withColumn('jytime',split(df_year['jysj']," ")[1])

df_year.show(2)

+-------------------+----------+--------+
|               jysj|      jyrq|  jytime|
+-------------------+----------+--------+
|2016-02-21 15:30:27|2016-02-21|15:30:27|
|2016-07-06 15:14:08|2016-07-06|15:14:08|
+-------------------+----------+--------+
only showing top 2 rows



In [214]:
df_year

DataFrame[jysj: string, jyrq: string, jytime: string]

In [215]:
from pyspark.sql.functions import month,year,dayofmonth,dayofweek,dayofyear
df_year=df_year.withColumn('year',year('jysj')).\
withColumn('month',month('jysj')).\
withColumn('day',dayofmonth('jysj')).\
withColumn('week',dayofweek('jysj')).\
withColumn('day_num',dayofyear('jysj'))  # h回去对于的年，月，日，一周内第几天，一年内第几天
df_year.show(3)

+-------------------+----------+--------+----+-----+---+----+-------+
|               jysj|      jyrq|  jytime|year|month|day|week|day_num|
+-------------------+----------+--------+----+-----+---+----+-------+
|2016-02-21 15:30:27|2016-02-21|15:30:27|2016|    2| 21|   1|     52|
|2016-07-06 15:14:08|2016-07-06|15:14:08|2016|    7|  6|   4|    188|
|2016-07-13 16:48:09|2016-07-13|16:48:09|2016|    7| 13|   4|    195|
+-------------------+----------+--------+----+-----+---+----+-------+
only showing top 3 rows



In [216]:
df_year.printSchema()

root
 |-- jysj: string (nullable = true)
 |-- jyrq: string (nullable = true)
 |-- jytime: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- day_num: integer (nullable = true)

